In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import logging
import gzip
import gensim 
import re
import spacy
import math
from collections import Counter
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import string
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence, pad_sequence
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [3]:
print(torch.cuda.is_available())
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device

True


'cuda:0'

# Preprocessing & Loading data

## Preprocess 

### Clothing Review Dataset

In [4]:
# Load dataset for clothing reviews


spam = pd.read_csv("/content/drive/MyDrive/data /spam.csv",encoding="latin1")
spam.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'],axis=1,inplace=True)
spam = spam[['v2', 'v1']]
spam.columns = ['content', 'detection']
spam['content_length'] = spam['content'].apply(lambda x: len(x.split()))
#reviews = pd.read_csv("/content/drive/MyDrive/data /Womens Clothing E-Commerce Reviews.csv")
#reviews = reviews.dropna()
#print(reviews.shape)
spam.head()

,content,detection,content_length
0,"Go until jurong point, crazy.. Available only ...",ham,20
1,Ok lar... Joking wif u oni...,ham,6
2,Free entry in 2 a wkly comp to win FA Cup fina...,spam,28
3,U dun say so early hor... U c already then say...,ham,11
4,"Nah I don't think he goes to usf, he lives aro...",ham,13


In [5]:
tracker = []
for aDetection in spam['detection']: 
  if not(aDetection in tracker): 
    tracker.append(aDetection)
print(len(tracker))

number_Detection = {}
index = 0
for aDetection in tracker: 
  number_Detection[aDetection] = index 
  index = index + 1
spam['detection'] = spam['detection'].apply(lambda x: number_Detection[x])
# reviews['Title'] = reviews['Title'].fillna('')
# reviews['Review Text'] = reviews['Review Text'].fillna('')
# reviews['review'] = reviews['Title'] + ' ' + reviews['Review Text']

2


In [6]:
spam.head()

,content,detection,content_length
0,"Go until jurong point, crazy.. Available only ...",0,20
1,Ok lar... Joking wif u oni...,0,6
2,Free entry in 2 a wkly comp to win FA Cup fina...,1,28
3,U dun say so early hor... U c already then say...,0,11
4,"Nah I don't think he goes to usf, he lives aro...",0,13


In [7]:
# reviews = reviews[['review', 'Rating']]
# reviews.columns = ['review', 'rating']
# reviews.head()
spam = spam[['content', 'detection']]
spam.columns = ['content', 'detection']
spam.head()

,content,detection
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0


#### Tokenize each sentence

In [8]:
#take advantage of nltk to tokenize all sentences
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
en_stop_words = set(stopwords.words('english'))
tokenizer = nltk.RegexpTokenizer(r'\w+|\$[\d\.]+')

def tokenize_sent(sent):
    
    tokenized = tokenizer.tokenize(sent)
    filtered = [w.lower() for w in tokenized if w.lower() not in en_stop_words]
    return filtered

spam['tokenized'] = spam['content'].apply(lambda x: tokenize_sent(x))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [9]:
# Sanity cehck for tokenizers
print(spam.head(3))

                                             content  ...                                          tokenized
0  Go until jurong point, crazy.. Available only ...  ...  [go, jurong, point, crazy, available, bugis, n...
1                      Ok lar... Joking wif u oni...  ...                     [ok, lar, joking, wif, u, oni]
2  Free entry in 2 a wkly comp to win FA Cup fina...  ...  [free, entry, 2, wkly, comp, win, fa, cup, fin...

[3 rows x 3 columns]


#### Encode and truncate sentence

In [10]:
# get unique words in the corpus
all_words = []
for x in spam['tokenized']:
    all_words.extend(x)

word_set = list(set(all_words))
word_count = Counter(all_words)

# filter out words with low frequency
for word_list in spam.tokenized:
    new_list = []
    for word in word_list:
        if word_count[word] > 2:
            new_list.append(word)
    word_list = new_list

# update set of words after removing the ones with low frequency
new_word_list = []
for x in spam['tokenized']:
    new_word_list.extend(x)
word_set = list(set(new_word_list))

# map each unique words & unknown token in reviews.encoded to an index
word2index = {}
word2index['<UNK>'] = 0
word2index['<PAD>'] = 1

for i, word in enumerate(word_set, 2):
    word2index[word] = i

# encode the original sequence
def encode(sent_list):
    result = []
    for x in sent_list:
        index = word2index[x]
        result.append(index)
    return result

spam['encoded'] = spam.tokenized.apply(lambda x: encode(x))

print(spam.head())

# get sequence average length
total_len = 0
for x in spam.encoded:
    total_len += len(x)

ave_len = math.floor(total_len/spam.shape[0])

# filter out long sequences --> encode all sequence to length = ave_len
# pad short sequence

for i, row in spam.iterrows():
    size = min(len(row.encoded), ave_len)
    new_encoded = row.encoded[:size]
    if size < ave_len:
        for j in range(0, ave_len - len(row.encoded)):
            new_encoded.append(1)
    spam.at[i, 'encoded'] = new_encoded

spam['review_length'] = spam.encoded.apply(lambda x: len(x))

                                             content  ...                                            encoded
0  Go until jurong point, crazy.. Available only ...  ...  [5495, 3528, 6040, 5140, 3679, 7997, 1147, 204...
1                      Ok lar... Joking wif u oni...  ...                [1540, 5113, 2546, 3599, 902, 1081]
2  Free entry in 2 a wkly comp to win FA Cup fina...  ...  [5276, 2817, 124, 3354, 5980, 3954, 1424, 583,...
3  U dun say so early hor... U c already then say...  ...  [902, 7083, 5534, 7810, 133, 902, 348, 2098, 5...
4  Nah I don't think he goes to usf, he lives aro...  ...         [7929, 6839, 8279, 7012, 8323, 6768, 6445]

[5 rows x 4 columns]


In [11]:
# sanity check for encoding:
print(len(all_words))
print(ave_len)
lengths = [len(x) for x in spam.encoded]
print(max(lengths))
print(min(lengths))
print(set(spam.detection))
review_length = spam.review_length
spam.head()
print(len(word_set))

53413
9
9
9
{0, 1}
8577


In [12]:
# Train test split from skearln
data_size = len(spam['encoded'])
assert data_size == len(spam['detection']) 
X, y = list(zip(list(spam['encoded']),(list(spam['review_length'])))), list(spam['detection'])
# X(data, length)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=43)

In [13]:
# Sanity check for length match
print(X_train[0])

([1455, 6765, 7966, 7557, 4925, 6765, 2324, 7215, 2466], 9)


## Dataset and Dataloader

### Clothing Review Dataset

In [14]:
class Dataset_Glove(Dataset):
    def __init__(self, X, y): 
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return  self.X[idx], self.y[idx]

In [15]:
trainSet_glv = Dataset_Glove(X_train, y_train)
testSet_glv = Dataset_Glove(X_test, y_test)

In [16]:
print(trainSet_glv[0])

(([1455, 6765, 7966, 7557, 4925, 6765, 2324, 7215, 2466], 9), 0)


In [17]:
def myCollate(batch):
    data = [item[0] for item in batch]
    label = [item[1] for item in batch]
    # sort sequence according to it's length in a batch
    data = data
    # data.sort(key=lambda x: x[1], reverse = True)

    # data (review: list, length: int)
    review = torch.tensor([x[0] for x in data], dtype = torch.long)
    label = torch.tensor(label, dtype = torch.long)
    seq_len = [x[1] for x in data]
    return review, label, seq_len

In [18]:
trainSet_glv = Dataset_Glove(X_train, y_train)
testSet_glv = Dataset_Glove(X_test, y_test)

In [19]:
trainLoader_glv = DataLoader(dataset = trainSet_glv, batch_size = 16, collate_fn = myCollate, shuffle = True, drop_last = True)
testLoader_glv = DataLoader(dataset = testSet_glv, batch_size = 16, collate_fn = myCollate, shuffle = True, drop_last = True)
it = iter(trainLoader_glv)
x, y, seq_len = it.next()

In [20]:
x.data

tensor([[1540, 5492, 7880, 4878, 3390, 7423, 4893, 5559, 3255],
        [ 902, 7880,  902, 2154, 4223, 4878,    1,    1,    1],
        [1812, 7880, 6982, 3033, 7166,    1,    1,    1,    1],
        [7661, 4857, 6602,  308, 3774, 2463, 1657, 3871, 4126],
        [3877,  940,   85,  738, 7230, 4415, 2248,  966,    1],
        [5070,   66, 4593, 1016,   47,    1,    1,    1,    1],
        [1540, 1923,  887,    1,    1,    1,    1,    1,    1],
        [3347, 1455, 3623, 4615, 3786, 4314,    1,    1,    1],
        [7494, 1506, 1181, 4834, 6072, 5100, 6730, 5410, 6703],
        [4792,  902, 8037, 6257,    1,    1,    1,    1,    1],
        [1766, 5558, 6370, 4777, 5641, 8232, 3230,    1,    1],
        [2708, 5781, 5449, 7122, 1447, 1929, 7122, 3242, 3539],
        [1887, 6125, 6885, 7852,  945, 1923,  887, 2681,  902],
        [2421, 1668,    1,    1,    1,    1,    1,    1,    1],
        [1789, 7375, 2543, 5741, 5087, 2789, 5822, 3374, 7883],
        [1946, 3583, 8276, 7364, 8022, 7

In [21]:
y

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0])

In [22]:
seq_len

[9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9]

## Embedding

In [23]:
def load_glove_vectors(glove_file="/content/drive/MyDrive/data /glove.6B.50d.txt"):
    """Load the glove word vectors"""
    word_vectors = {}
    with open(glove_file) as f:
        for line in f:
            split = line.split()
            word_vectors[split[0]] = np.array([float(x) for x in split[1:]])
    return word_vectors


In [24]:
# create embedding matrix
def emb_matrix(model, all_words, word2index):
    matrix_size = len(word_set) + 2
    emb_matrix = np.zeros((matrix_size, 50))
    emb_matrix[0] = np.random.uniform(-0.25, 0.25, 50) # vector for UNK
    emb_matrix[1] = np.zeros(50) # vector for padding, has no weight

    # vector for every other word in the dictionary
    for i in range(2, len(all_words)+2):
        word = all_words[i-2]
        index = word2index[word]
        try:
            vector = model[word]
        except Exception as e:
            # word does not exist in the pretrained embedding
            vector = emb_matrix[0]
        emb_matrix[index] = vector
    return emb_matrix

In [25]:
word2vec = load_glove_vectors()
data = list(word2vec.values())
an_array=np.array(data)

In [26]:
embedding = emb_matrix(word2vec, word_set, word2index)

In [27]:
print(embedding.shape)

(8579, 50)


In [28]:
print(embedding)

[[ 0.23951224 -0.2269762   0.20430948 ...  0.07038463  0.02220542
   0.12337086]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 1.3249     -0.72295    -1.4014     ...  0.15883     0.8059
  -0.72778   ]
 ...
 [-1.0142      0.7214      0.34739    ... -0.40096    -0.10437
  -0.35435   ]
 [ 0.89453    -0.30377     0.21361    ...  0.85475    -0.14677
   0.2617    ]
 [ 0.34764    -0.24896     0.27224    ... -0.28589     0.06025
   0.36345   ]]


In [29]:
def validation_metrics(model, valid_dl):
    print("current in vlaidaiton")
    model.eval()
    correct = 0
    total = 0
    sum_loss = 0.0
    sum_rmse = 0.0
    criterion = nn.CrossEntropyLoss()
    model = model
    for x, y, seq_len in valid_dl:
        x = x.long()
        y = y.long()
        y_hat = model(x, seq_len)
        loss = criterion(y_hat, y)
        
        # print(f"shape of y: {y.shape}")
        # print(f"shape of y_pred: {y_hat.shape}")
        # print(y_hat)
        pred = torch.argmax(y_hat, 1)
        # print(pred)
        correct += sum((pred == y))
        total += y.shape[0]
        sum_loss += loss.item()*y.shape[0]

    return sum_loss/total, correct/total

In [30]:
def train_model(model, fname, epochs = 50, lr = 0.0001):
    parameters = filter(lambda p:p.requires_grad, model.parameters())
    optimizer = torch.optim.Adam(parameters, lr = lr)
    criterion = nn.CrossEntropyLoss()
    model = model
    PATH = '/content/drive/MyDrive/McGill/Comp 550/' + fname
    epoch = 0
    best_val_acc = 0.0
    for i in range(epochs):
        epoch += 1
        print(f"At epoch {i}")
        batch_count = 0
        model.train()
        sum_loss = 0.0
        total = 0.0
        train_correct = 0.0
        for x, y, seq_len in tqdm(trainLoader_glv):
            x = x.long()
            y = y.long()
            batch_count += 1
            optimizer.zero_grad()
            y_pred = model(x, seq_len)
            pred = torch.argmax(y_pred, 1)
            train_correct += sum((pred == y))
            loss = criterion(y_pred, y)
            loss.backward()
            optimizer.step()
            sum_loss += loss.item()*y.shape[0]
            total += y.shape[0]
        val_loss, val_acc = validation_metrics(model, testLoader_glv)
        print("train loss %.3f, train accuracy %.3f, val loss %.3f, val accuracy %.3f" 
              % (sum_loss/total, train_correct/total, val_loss, val_acc))
        if val_acc > best_val_acc and i>=2:
            best_val_acc = val_acc
            NEW_PATH = PATH+f'_{i}_{val_acc}'
            #torch.save(model.state_dict(), NEW_PATH)
            print(f"\t=> Best model saved at {i}th epoch with valication accuracy of {val_acc}")

In [31]:
weight = torch.FloatTensor(an_array)
word2vec_embedding = nn.Embedding.from_pretrained(weight)

class LSTM_word2vec(torch.nn.Module):
    def __init__(self, emb_matrix, embedding_dim, hidden_dim):
        super().__init__()
        self.embedding_dim = embedding_dim
        self.embeddings = nn.Embedding(emb_matrix.shape[0], embedding_dim, padding_idx = 0)
        self.embeddings.weight.data.copy_(torch.from_numpy(emb_matrix))
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.linear = nn.Linear(hidden_dim, 5)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x, seq_len):
        x = self.embeddings(x)
        # x = nn.utils.rnn.pack_padded_sequence(x, self.embedding_dim, batch_first = True)
        x = self.dropout(x)
        # x_padded = pack_padded_sequence(x, seq_len, batch_first = True)
        output, (h_n, c_n) = self.lstm(x)
        # print(h_n[-1].shape)
        # output (N, ave_len, num_class)
        # reshape to out (N, num_class)
        
        return self.linear(h_n[-1])

In [32]:
model = LSTM_word2vec(embedding, 50, 100)
train_model(model, fname='word2vec_lstm', epochs=10)

At epoch 0


100%|██████████| 278/278 [00:01<00:00, 148.46it/s]


current in vlaidaiton
train loss 0.789, train accuracy 0.853, val loss 0.382, val accuracy 0.861
At epoch 1


100%|██████████| 278/278 [00:01<00:00, 176.58it/s]


current in vlaidaiton
train loss 0.289, train accuracy 0.888, val loss 0.261, val accuracy 0.888
At epoch 2


...

current in vlaidaiton
train loss 0.140, train accuracy 0.947, val loss 0.141, val accuracy 0.947
At epoch 8


100%|██████████| 278/278 [00:01<00:00, 157.01it/s]


current in vlaidaiton
train loss 0.130, train accuracy 0.953, val loss 0.135, val accuracy 0.948
	=> Best model saved at 8th epoch with valication accuracy of 0.948369562625885
At epoch 9


100%|██████████| 278/278 [00:01<00:00, 165.68it/s]


current in vlaidaiton
train loss 0.128, train accuracy 0.955, val loss 0.134, val accuracy 0.948
